# Import and setting

In [ ]:
!pwd

In [ ]:
#---------- for python2
from __future__ import print_function
from __future__ import division

In [ ]:
#---------- import
import gzip
import pickle

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from matplotlib.animation import FuncAnimation
import numpy as np

%matplotlib inline

In [ ]:
# ---------- figure
mpl.rcParams['figure.figsize'] =[8, 6]

# ---------- axes
mpl.rc('axes', linewidth=1.5, grid=True)

# ---------- ticks
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
mpl.rc('xtick.major', width=1.0, size=8.0)
mpl.rc('ytick.major', width=1.0, size=8.0)

# ---------- lines
mpl.rcParams['lines.linewidth'] = 2.5

# ---------- grid
mpl.rcParams['grid.linestyle'] = ':'

# ---------- font
mpl.rcParams['font.size'] = 16
mpl.rcParams['pdf.fonttype'] = 42    # embed fonts in PDF using type42 (True type)

# Load data

In [ ]:
with open('./pkl_data/rslt_data.pkl', 'rb') as f:
        rslt_data = pickle.load(f)

In [ ]:
with open('./pkl_data/LAQA_id_data.pkl', 'rb') as f:
        id_to_calc, id_select_hist, id_done = pickle.load(f)

In [ ]:
with open('./pkl_data/LAQA_data.pkl', 'rb') as f:
        total_step, LAQA_step, LAQA_struc, LAQA_energy, LAQA_bias, LAQA_score = pickle.load(f)

In [ ]:
#  ---------- If you used gzip
with gzip.open('./pkl_data/LAQA_data.pkl.gz', 'rb') as f:
        total_step, LAQA_step, LAQA_struc, LAQA_energy, LAQA_bias, LAQA_score = pickle.load(f)

# Results

## cryspy_rslt

In [ ]:
# ---------- top 10
rslt_data.sort_values(by=['Energy']).head(10)

In [ ]:
stable_IDs = [13, 91, 48]    # manual input

## Required optimization steps

In [ ]:
# only completed structures
req_step = {}
tot_req_step = 0
for cID in range(len(LAQA_step)):
    if cID in id_done:
        req_step[cID] = sum(LAQA_step[cID])
        tot_req_step += sum(LAQA_step[cID])

In [ ]:
print('Total optimization steps:', sum(total_step))
print('Number of completed structures:', len(req_step))
print('Average number of optimization steps:', tot_req_step/len(req_step))

In [ ]:
for cid in stable_IDs:    # input 'stable_IDs' in an upper cell
    print('\nFor ID {}'.format(cid))
    for i, ids in enumerate(id_select_hist):
        if cid in ids:
            print('    Selection {}'.format(i+1))
    if cid in id_done:
        print('Opt. finished. # of steps ID {} is {}'.format(cid, sum(LAQA_step[cid])))
    else:
        print('Error')

In [ ]:
# ---------- total number of steps up to selection xxx
nselect = range(10)
#nselect = [0, 1, 31]
print('Total number of steps')
for n in nselect:
    print('Selection {}: {}'.format(n, sum(total_step[:n+1])))

## Energy vs. step for png figure

In [ ]:
# ---------- figsize
plt.figure(figsize=(10, 6))

# ---------- axes and ticks
# ------ x axis
plt.xlim([20, 280])
major_x = MultipleLocator(40)
minor_x = MultipleLocator(20)
plt.gca().xaxis.set_major_locator(major_x)
plt.gca().xaxis.set_minor_locator(minor_x)
# ------ yaxis
plt.ylim([-5.5, -4])
# ---------- gird
plt.grid(which='minor')    # grid: major --> minor

# ---------- title and label
plt.title('LAQA for Si$_{16} (c = 1.0)$ ')
plt.xlabel('Number of step')
plt.ylabel('Energy (eV/atom)')
plt.tight_layout()

# ---------- plot
# ------ not stalbe structures
for cID in range(len(LAQA_energy)):
    if cID == stable_IDs:
        pass
    else:
        plt.plot(np.cumsum(LAQA_step[cID]), LAQA_energy[cID], color='royalblue', linewidth=1.0)
# ------ stable structures, bring to front
for cID in stable_IDs:
    plt.plot(np.cumsum(LAQA_step[cID]), LAQA_energy[cID], color='red')

# ---------- save figure
plt.savefig('Si16_LAQA_c1.0.png', bbox_inches='tight')    # PNG

## Energy gif anime

In [ ]:
# ---------- for animation in jupyter
# If you use nbagg, you can see animation on Jupyter
# When you want to save the animation, do NOT use %matplotlib nbagg
#%matplotlib nbagg
#%matplotlib inline

# ---------- figure setting
fig = plt.figure(figsize=(10, 6))
ax = plt.axes()


# --------- initialize
lines = []
xdata = []
ydata = []
num_select = {}
for cID in range(len(LAQA_energy)):
    xdata.append([LAQA_step[cID][0]])
    ydata.append([LAQA_energy[cID][0]])
    num_select[cID] = 0
    if cID in stable_IDs:
        lines.append(ax.plot([], [], color='red', linewidth=2.0)[0])
    else:
        lines.append(ax.plot([], [], color='royalblue', linewidth=1.0)[0])

def init():
    ax.set_xlim([20, 280])
    major_x = MultipleLocator(40)
    minor_x = MultipleLocator(20)
    ax.xaxis.set_major_locator(major_x)
    ax.xaxis.set_minor_locator(minor_x)
    plt.grid(which='minor')    # grid: major --> minor
    ax.set_ylim([-5.5, -4])
    plt.title('LAQA for Si$_{16} (c = 1.0)$ ')
    ax.set_xlabel('Number of step')
    ax.set_ylabel('Energy (eV/atom)')
    fig.tight_layout()
    return lines

# ---------- animate function
# frame i --> 0, 1, 2, ...
def animate(i):
    for cID in id_select_hist[i]:
        num_select[cID] += 1
        xdata[cID].append(LAQA_step[cID][num_select[cID]])
        ydata[cID].append(LAQA_energy[cID][num_select[cID]])
        lines[cID].set_data(np.cumsum(xdata[cID]), ydata[cID])
    return lines


# ---------- call the animator
# ------ if the simulation was done
#anim = FuncAnimation(fig, animate, init_func=init, frames=len(id_select_hist), interval=100, blit=True)
# ------ else, last id list in id_select_hist may not be calculated yet, i.e. remains in id_to_calc
anim = FuncAnimation(fig, animate, init_func=init, frames=len(id_select_hist)-1, interval=200, blit=True)

In [ ]:
# ---------- save
anim.save('Si16_LAQA_c1.0.gif',  writer = 'imagemagick')

 ![]('./Si16_LAQA_energy_c1.0.gif')

Show gif anime  
Specify a suitable filename  
![](Si16_LAQA_energy_c1.0.gif)